## Домашнее задание: "Детекция объектов на изображении"

ФИО:

## Цель задания:
Научиться самостоятельно решать задачу детекции.
## Описание задания:
В рамках данного домашнего задания предлагается решить задачу детекции мячей и настроить полный пайплайн обучения модели.

Процесс выполнения задания следующий:

0. Выбор модели детекции для обучения:
    1. Выберите модель детекции для выполнения домашнего задания. Любую, кроме RetinaNet. Ее реализацию можно взять из открытых источников. Модель можно брать предобученную ( в этом случае в пункте 4. показать влияние предобучения на финальное качество).
    2. Полезные ссылки: [PyTorch Vision Models](https://pytorch.org/vision/stable/models.html) (блок Object Detection), [SOTA модели детекции](https://paperswithcode.com/sota/object-detection-on-coco), [Возможный пример кода](https://github.com/AlekseySpasenov/dl-course/blob/autumn_2023/lecture8/detection_example/pytorch_detection_workshop.ipynb)
    3. Вы можете использовать RetinaNet, которая была реализована на семинаре, но это приведет к снижению оценки на **–2.5 балла**, так как задания 1.1 и 2.1 уже были выполнены в рамках занятия.

1. Подготовка обучающего набора данных
    0. Для выполнения задания используйте датасет с изображениями мячей, который использовался на семинаре.
    1. Реализуйте корректный класс Dataset и Dataloader для выбранной модели (должен работать форвард вашей модели на том, что выходит из даталоадера) **0.5 балла**.
    2. Добавьте простые аугментации в датасет (аугментации, не затрагивающие изменение ground-truth bounding box) **0.5 балла**.
    3. Внедрите сложные аугментации (аугментации, затрагивающие изменение ground-truth bounding box. Например, аффинные преобразования: сдвиг, поворот и т.д.) **0.5 балла**.

    4. Полезные ссылки: https://pytorch.org/vision/stable/transforms.html , https://albumentations.ai

2. Реализация корректного train-loop и обучение модели:  
    1. Реализуйте эффективный train-loop для вашей модели и проведите обучение **2 балла**.
    2. Выполните несколько запусков обучения с различными параметрами, например: сравните влияние различных аугментаций, оцените влияние того была предобучена модель или нет, сравните результаты при изменении гиперпараметров итд (на ваш выбор) **0.5 балла**.

3. Валидация обученных моделей на тестовой выборке, вычисление метрики mAP
    1. Оцените качество моделей на тестовой части данных и рассчитайте метрику mAP **0.5 балл**
    2. Полезные ссылки: [mean_average_precision](https://github.com/bes-dev/mean_average_precision)

4. Выводы **0.5 балл**:
    1. Проанализируйте результаты обучения, визуально оцените качество работы модели.
    2. Прокомментируйте распространенные ошибки модели и предложите пути для улучшения финального решения.

In [1]:
import torch
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2
import os
import matplotlib.pyplot as plt
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torch.cuda.amp import GradScaler, autocast
from tqdm import tqdm

In [2]:
base_dir = 'seminar_objdet_retina_oi5_ball'
image_dir = os.path.join(base_dir, 'oi5_ball')
train_json_path = os.path.join(base_dir, 'oi5_ball_filename_to_bbox_train.json')
val_json_path = os.path.join(base_dir, 'oi5_ball_filename_to_bbox_val.json')

with open(train_json_path, 'r') as f:
    train_annotations = json.load(f)
with open(val_json_path, 'r') as f:
    val_annotations = json.load(f)

annotations = {**train_annotations, **val_annotations}
train_images = list(train_annotations.keys())
val_images = list(val_annotations.keys())

np.random.seed(42)
np.random.shuffle(val_images)
val_size = int(0.7 * len(val_images))
val_images, test_images = val_images[:val_size], val_images[val_size:]

In [3]:
class BallDataset(Dataset):
    def __init__(self, image_list, annotations, image_dir, transforms=None, is_train=True):
        self.image_list = image_list
        self.annotations = annotations
        self.image_dir = image_dir
        self.transforms = transforms
        self.is_train = is_train

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_list[idx].split('/')[-1])
        img = Image.open(img_path).convert("RGB")
        width, height = img.size

        bboxes = self.annotations.get(self.image_list[idx], [])
        abs_bboxes = []
        for bbox in bboxes:
            xmin = bbox[0] * width
            ymin = bbox[2] * height
            xmax = bbox[1] * width
            ymax = bbox[3] * height
            x_min = min(xmin, xmax)
            y_min = min(ymin, ymax)
            x_max = max(xmin, xmax)
            y_max = max(ymin, ymax)
            if x_max > x_min and y_max > y_min:
                abs_bboxes.append([x_min, y_min, x_max, y_max])

        num_objs = len(abs_bboxes)
        labels = [1] * num_objs

        if self.transforms and self.is_train and num_objs > 0:
            img_np = np.array(img)
            transformed = self.transforms(image=img_np, bboxes=abs_bboxes, class_labels=labels)
            img = transformed['image'].float() / 255.0
            abs_bboxes = transformed['bboxes']
            labels = transformed['class_labels']
        else:
            img = torch.as_tensor(np.array(img), dtype=torch.float32).permute(2, 0, 1) / 255.0

        target = {
            'boxes': torch.as_tensor(abs_bboxes, dtype=torch.float32) if abs_bboxes else torch.zeros((0, 4)),
            'labels': torch.as_tensor(labels, dtype=torch.int64) if labels else torch.zeros(0),
            'image_id': torch.tensor([idx])
        }
        return img, target

In [4]:
simple_transforms = A.Compose([
    A.RandomBrightnessContrast(p=0.5),
    A.HueSaturationValue(p=0.5),
    A.GaussianBlur(p=0.3),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

complex_transforms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=30, p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['class_labels']))

c:\Users\101\source\repos\bmstu_basic_nn_2025\.venv\Lib\site-packages\albumentations\core\composition.py:331: UserWarning: Got processor for bboxes, but no transform to process it.
  self._set_keys()
c:\Users\101\source\repos\bmstu_basic_nn_2025\.venv\Lib\site-packages\albumentations\core\validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [5]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [6]:
train_dataset_simple = BallDataset(train_images, annotations, image_dir, transforms=simple_transforms, is_train=True)
train_dataset_complex = BallDataset(train_images, annotations, image_dir, transforms=complex_transforms, is_train=True)
val_dataset = BallDataset(val_images, annotations, image_dir, transforms=ToTensorV2(), is_train=False)
test_dataset = BallDataset(test_images, annotations, image_dir, transforms=ToTensorV2(), is_train=False)

batch_size = 16
train_loader_simple = DataLoader(train_dataset_simple, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=1)
train_loader_complex = DataLoader(train_dataset_complex, batch_size=batch_size, shuffle=True, collate_fn=collate_fn, num_workers=1)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=1, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn, num_workers=1, pin_memory=True)

In [7]:
def get_model(pretrained=True):
    model = ssdlite320_mobilenet_v3_large(weights=SSDLite320_MobileNet_V3_Large_Weights.COCO_V1)
    num_classes = 2  # ball + background
    num_anchors = len(model.anchor_generator.aspect_ratios[0])  # 6 для SSDLite
    
    # Обновление classification_head
    for i, seq in enumerate(model.head.classification_head.module_list):
        if len(seq) >= 2 and isinstance(seq[1], torch.nn.Conv2d):
            in_channels = seq[0][0].in_channels  # Извлекаем in_channels из первого слоя (Conv2dNormActivation)
            # Заменяем второй слой на новый Conv2d для 2 классов
            model.head.classification_head.module_list[i][1] = torch.nn.Conv2d(
                in_channels, num_anchors * num_classes, kernel_size=3, padding=1
            )
    
    model.to(device)
    return model

In [8]:
scaler = GradScaler()
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    running_loss = 0.0
    num_batches = len(data_loader)
    
    with tqdm(total=num_batches, desc=f'Epoch {epoch+1}/{10}', unit='batch') as pbar:
        for i, (images, targets) in enumerate(data_loader):
            valid_pairs = [(img, tgt) for img, tgt in zip(images, targets) if tgt['boxes'].shape[0] > 0]
            if not valid_pairs:
                pbar.update(1)
                continue
            images = [pair[0].to(device) for pair in valid_pairs]
            targets = [{k: v.to(device) for k, v in pair[1].items()} for pair in valid_pairs]
            
            optimizer.zero_grad()
            with autocast():
                loss_dict = model(images, targets)
                losses = sum(loss for loss in loss_dict.values())
            
            scaler.scale(losses).backward()
            scaler.step(optimizer)
            scaler.update()
            
            batch_loss = losses.item()
            running_loss += batch_loss
            
            vram_allocated = torch.cuda.memory_allocated() / 1024**2  # MB
            vram_total = torch.cuda.get_device_properties(0).total_memory / 1024**2  # MB
            vram_used = vram_total - torch.cuda.memory_reserved() / 1024**2  # MB
            
            pbar.set_postfix({'Loss': f'{batch_loss:.4f}', 
                             'VRAM Used': f'{vram_used:.1f}MB/{vram_total:.1f}MB'})
            pbar.update(1)
    
    avg_loss = running_loss / num_batches if num_batches > 0 else 0.0
    print(f"Epoch {epoch+1} completed. Average Loss: {avg_loss:.4f}")
    return avg_loss

def train_model(model, train_loader, num_epochs=10, lr=0.005):
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.0005)
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    losses = []
    for epoch in range(num_epochs):
        loss = train_one_epoch(model, optimizer, train_loader, device, epoch)
        losses.append(loss)
        lr_scheduler.step()
        torch.save(model.state_dict(), f'model_epoch_{epoch}.pth')
    return losses


C:\Users\101\AppData\Local\Temp\ipykernel_8968\308018599.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
print("Number of batches:", len(train_loader_complex))
for i, (images, targets) in enumerate(train_loader_complex):
    print(f"Batch {i+1}, Images shape: {images[0].shape if images else 'None'}, Targets: {targets[0]['boxes'].shape if targets[0]['boxes'].numel() > 0 else 'Empty'}")
    if i == 2: break

Number of batches: 180


In [ ]:
model1 = get_model(pretrained=True)
losses1 = train_model(model1, train_loader_complex)
plt.plot(losses1, label='Pretrained + Complex Aug')
plt.legend()
plt.show()

Epoch 1/10:   0%|          | 0/180 [00:00<?, ?batch/s]

In [ ]:
model2 = get_model(pretrained=False)
losses2 = train_model(model2, train_loader_complex)
plt.plot(losses2, label='Scratch + Complex Aug')
plt.legend()
plt.show()

In [ ]:
model3 = get_model(pretrained=True)
losses3 = train_model(model3, train_loader_simple)
plt.plot(losses3, label='Pretrained + Simple Aug')
plt.legend()
plt.show()

In [ ]:
@torch.no_grad()
def evaluate(model, data_loader, device):
    metric = MeanAveragePrecision(iou_thresholds=[0.5])
    model.eval()
    for images, targets in data_loader:
        valid_pairs = [(img, tgt) for img, tgt in zip(images, targets) if tgt['boxes'].shape[0] > 0]
        if not valid_pairs:
            continue
        images = [img.to(device) for img, _ in valid_pairs]
        tgts = [{k: v.to(device) for k, v in tgt.items()} for _, tgt in valid_pairs]
        outputs = model(images)
        metric.update(outputs, tgts)
    return metric.compute()['map'].item()

models = [model1, model2, model3]
model_names = ['Pretrained + Complex', 'Scratch + Complex', 'Pretrained + Simple']

for i, model in enumerate(models):
    map_val = evaluate(model, val_loader, device)
    map_test = evaluate(model, test_loader, device)
    print(f"{model_names[i]} - Val mAP: {map_val}, Test mAP: {map_test}")

In [ ]:
def visualize(img, target, pred):
    img = img.permute(1, 2, 0).cpu().numpy()
    plt.imshow(img)
    for box in target['boxes']:
        plt.gca().add_patch(plt.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], fill=False, color='green'))
    for i, box in enumerate(pred['boxes']):
        if pred['scores'][i] > 0.5:
            plt.gca().add_patch(plt.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], fill=False, color='red'))
    plt.axis('off')
    plt.show()

img, target = test_dataset[0]
model1.eval()
pred = model1([img.to(device)])[0]
visualize(img, target, pred)

### Оставьте обратную связь по занятию №6
Это поможет улучшить курс и сделать следующие занятия ещё лучше!
Форма обратной связи: https://forms.gle/hTfRQaGBrqic7LyP6


In [ ]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))
print("VRAM allocated:", torch.cuda.memory_allocated() / 1024**2, "MB")

CUDA available: True
GPU name: NVIDIA GeForce RTX 3070
VRAM allocated: 0.0 MB
